**Notebook content:**
- 
- 

In [127]:
%reset -f 

In [128]:
import sys
import pandas as pd
import numpy as np
import datetime
import json
from IPython.display import display

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 200)
#pd.set_option('display.float_format', lambda x: '%.f' % x)
pd.options.display.precision = 4

#import sys
np.set_printoptions(threshold=sys.maxsize) #- print the full NumPy array

from myDefs.defs import *

# visualization
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
%matplotlib inline


**Read events file**

In [129]:
#read events
path = DATA_PATH + "parseData1.csv"
df = pd.read_csv(path, sep=',') 
#parse date of surgery
df['eventStartDate'] = pd.to_datetime(df['eventStartDate'], format='%Y-%m-%d')
df['eventEndDate'] = pd.to_datetime(df['eventEndDate'], format='%Y-%m-%d')

#read annonymous file
path = "{}parseAnonymous0.csv".format(DATA_PATH)
anonymous = pd.read_csv(path, sep=',')
#parse date of surgery
anonymous['Date of surgery'] = pd.to_datetime(anonymous['Date of surgery'], format='%Y-%m-%d')


In [130]:
#read events info
eventsinfo = pd.read_excel(EVENTS_INFO_PATH + "EventsInfo.xlsx")

In [131]:
print(df.shape)
print(anonymous.shape)
print(eventsinfo.shape)

(1683616, 17)
(718, 14)
(3755, 15)


# New column: feature name

In [132]:
#create new column feature name 
names_dict = eventsinfo.loc[eventsinfo['feature_name'].notna(), :]
names_dict = dict(zip(names_dict['eventName'], names_dict['feature_name']))

df['feature_name'] = df['eventName']
df['feature_name'] = df['feature_name'].replace(names_dict)

In [133]:
df['feature_name'].nunique()

2927

# Replace values (sValue -> dValue), create column "numeric"

In [134]:
def convert_to_dict(x):
    s = x.replace('""', '"')
    d = json.loads(s)
    for key in d:
        if d[key] == 'None':
            d[key] = None
    return d

#replace dValue by values in dictionary
def replace_values(x, info):
    #get dictionary 
    d = info.loc[info['eventName'] == x['eventName'].iloc[0], 'dict'].iloc[0]
    #copy sValue to dValue
    x.loc[x['sValue'].notna(),'dValue'] = x['sValue']
    #replace dValues by by sValues 
    return x['dValue'].replace(d)

#convert dict column to dictionary 
eventsinfo.loc[eventsinfo['dict'].notna(),'dict'] = eventsinfo.loc[eventsinfo['dict'].notna(),'dict'].apply(convert_to_dict)

#replace values
df['numeric'] = df.groupby('eventName', group_keys=False).apply(replace_values, eventsinfo)

# Create new column "unit"

In [135]:
#create new column of unit
df['unit'] = df['unitOfMeasure']

## reorder columns

In [136]:
df = df[['pid', 'eventName', 'feature_name','eventStartDate', 'eventEndDate',  'numeric', 'unit',
       'dValue', 'iValue', 'sValue', 'eventDesc', 'unitOfMeasure',
       'orderNumber', 'organismId', 'eventCode', 'eventCodeOrg', 'eventType',
       'eventTypeOrg', 'sourceName']]

In [137]:
df.columns

Index(['pid', 'eventName', 'feature_name', 'eventStartDate', 'eventEndDate',
       'numeric', 'unit', 'dValue', 'iValue', 'sValue', 'eventDesc',
       'unitOfMeasure', 'orderNumber', 'organismId', 'eventCode',
       'eventCodeOrg', 'eventType', 'eventTypeOrg', 'sourceName'],
      dtype='object')

# Filter events and slice data

In [138]:
#get only patient with a surgery date, which are not duplicate (first surgey)
anonymous_ix = anonymous[(anonymous['Date of surgery'].notna()) 
                         & (~anonymous['pid'].duplicated())].index
print("anonymous:", anonymous.loc[anonymous_ix].shape)

anonymous: (525, 14)


In [139]:
#filter events by thier info
#filter 1
filter1 = ((eventsinfo['category'] == 'laboratory') & (eventsinfo['inMatrix'] == 1) \
         & (eventsinfo['nPatients'] > 300))

filtered = eventsinfo.loc[filter1, 'dict'].dropna().head().replace()

#filter 2
filter2 = ((eventsinfo['category'] == 'physical') & \
        (eventsinfo['inMatrix'] == 1) & \
           
        (eventsinfo['nPatients'] > 30)) 

#filter 3
filter3 = ((eventsinfo['category'] == 'drug') & \
        (eventsinfo['inMatrix'] == 1) & \
            (eventsinfo['sValue'].isna()) & \
        (eventsinfo['nPatients'] > 30)) 

            
info_ix = eventsinfo[filter1 | filter2].index


info_ix.shape
#'Fentanyl'

(67,)

In [140]:
events_names = eventsinfo.loc[filter1 | filter2, 'eventName']
events_names;

In [141]:
#slice events data  
df_ix = df[df['eventName'].isin(eventsinfo.loc[info_ix, 'eventName']) ].index
print("df:", df.loc[df_ix].shape)


df: (743328, 19)


In [142]:
#get meta data from anonymous 

metadata = anonymous.loc[anonymous_ix]

#map: 1 if Anastomotic Leak, 0 o.w.
metadata['Anastomotic Leak'] = 0
metadata.loc[metadata['Complications']=='Anastomotic Leak', 'Anastomotic Leak'] = 1

#drop columns
cols_to_remove = ['Date of surgery', 'Days of hospitalization', 'Patient classification', 
                  'Responsible surgeon', 'Kk', 'Simple', 'Severe', 'Complications']
metadata = metadata.drop(columns=cols_to_remove).set_index('pid')

# convert units

In [143]:
#see which events should be converted
eventsinfo.loc[info_ix].loc[eventsinfo['convertUnit'].notna(), ['eventName', 'feature_name', 'unit']]

,eventName,feature_name,unit
15,Temperature,body temperature,['F']
16,חום,body temperature,"['F', 'C']"
251,WBC,WBC,"[nan, '/HPF', 'K/microL', 'cells/mm3', 'LEU/UL']"


#### Body temperature

In [144]:
def fahrenheit_to_celsius(f):
    return (f - 32) * 5.0/9.0

temp_ix = df.loc[df['feature_name'] == 'body temperature'].loc[df['unitOfMeasure'] == 'F'].index
df.loc[temp_ix, 'numeric'] = df.loc[temp_ix, 'numeric'].apply(fahrenheit_to_celsius)
df.loc[temp_ix, 'unit'] = 'C'

#remove noise
temp = df.loc[df['feature_name'] == 'body temperature', 'numeric']
temp_ix = temp[temp > 45].index
df.loc[temp_ix, 'numeric'] = None

#### WBC

In [145]:
wbc = df.loc[df['eventName'] == 'WBC', 'unitOfMeasure'].value_counts()
display(wbc)

for unit in wbc.head(3).index:
    wbc_ix = df.loc[df['feature_name'] == 'WBC'].loc[df['unitOfMeasure'] == unit].index
    df.loc[wbc_ix, 'feature_name'] = 'WBC_' + unit
    df.loc[wbc_ix, 'unit'] = unit



K/microL     14626
/HPF           468
LEU/UL         137
cells/mm3        5
Name: unitOfMeasure, dtype: int64

In [165]:

df.loc[df['eventName'] == 'LAPAROSCOPIC PARTIAL GASTRECTOMY WEDGE RESECTION 436103']

,pid,eventName,feature_name,eventStartDate,eventEndDate,numeric,unit,dValue,iValue,sValue,eventDesc,unitOfMeasure,orderNumber,organismId,eventCode,eventCodeOrg,eventType,eventTypeOrg,sourceName
233571,1.0175e+09,LAPAROSCOPIC PARTIAL GASTRECTOMY WEDGE RESECTI...,LAPAROSCOPIC PARTIAL GASTRECTOMY WEDGE RESECTI...,2018-02-18,2018-02-18,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,87604009,43.89,snomed,conv_proc_cham,[MIRROR_Chameleon].[Procedures]
495427,1.1283e+06,LAPAROSCOPIC PARTIAL GASTRECTOMY WEDGE RESECTI...,LAPAROSCOPIC PARTIAL GASTRECTOMY WEDGE RESECTI...,2018-09-25,2018-09-25,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,87604009,43.89,snomed,conv_proc_cham,[MIRROR_Chameleon].[Procedures]


In [173]:




df.loc[df['sourceName'] == '[MIRROR_Chameleon].[MedExecutionMedicine]']

,pid,eventName,feature_name,eventStartDate,eventEndDate,numeric,unit,dValue,iValue,sValue,eventDesc,unitOfMeasure,orderNumber,organismId,eventCode,eventCodeOrg,eventType,eventTypeOrg,sourceName
79,1.0133e+09,FLAGYL vial 500 mg,FLAGYL,2018-02-16 02:00:00,2018-02-16 02:00:00,500,mg,500.0,0.0,NaN,NaN,mg,NaN,NaN,317428,113730,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
104,1.0133e+09,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,2018-02-16 03:41:00,2018-02-16 03:41:00,50,mg,50.0,0.0,NaN,NaN,mg,NaN,NaN,6960,114086,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
105,1.0133e+09,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,2018-02-16 03:41:00,2018-02-16 03:41:00,50,cc,50.0,0.0,NaN,NaN,cc,NaN,NaN,1162505,146186,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
110,1.0133e+09,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,2018-02-16 03:42:00,2018-02-16 03:42:00,50,mg,50.0,0.0,NaN,NaN,mg,NaN,NaN,6960,114086,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
111,1.0133e+09,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,MIDAZOLAM 50 mg (6mg/hour)+SODIUM CHLORIDE 0.9...,2018-02-16 03:42:00,2018-02-16 03:42:00,50,cc,50.0,0.0,NaN,NaN,cc,NaN,NaN,1162505,146186,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683491,5.7390e+08,ROCEPHIN amp 1000 mg,ROCEPHIN,2018-09-13 01:11:00,2018-09-13 01:11:00,1000,mg,1000.0,0.0,NaN,NaN,mg,NaN,NaN,1665020,74231,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
1683592,1.0726e+08,DISOTHIAZIDE tab tab 25 mg,DISOTHIAZIDE,2018-10-21 17:33:00,2018-10-21 17:33:00,25,mg,25.0,0.0,NaN,NaN,mg,NaN,NaN,C03AA,100114,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
1683593,1.0726e+08,SIMVASTATIN-TEVA tab tab 10 mg,SIMVASTATIN-TEVA,2018-10-21 17:33:00,2018-10-21 17:33:00,10,mg,10.0,0.0,NaN,NaN,mg,NaN,NaN,C10AA,145736,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]
1683594,1.0726e+08,DISOTHIAZIDE tab tab 25 mg,DISOTHIAZIDE,2018-10-21 17:33:00,2018-10-21 17:33:00,25,mg,25.0,0.0,NaN,NaN,mg,NaN,NaN,C03AA,100114,rxnorm,conv_medicine_cham,[MIRROR_Chameleon].[MedExecutionMedicine]


In [190]:
df.loc[df['feature_name'] == 'body temperature', 'eventCode'].value_counts()

x = df.groupby('eventCode', group_keys=False)['eventName'].size().sort_values()

df.loc[df['eventCode'] == '271649006', 'eventName'].value_counts()

ArtBPS          35468
לחץ סיסטולי     28682
NiBPS            4419
ART Systolic      669
Name: eventName, dtype: int64

#### weight

In [147]:
#convert to kg 
weight_ix = df.loc[df['eventName'] == 'CurrentWeight'].index
df.loc[weight_ix, 'numeric'] = df.loc[weight_ix, 'numeric'].apply(lambda x: x/1000)
df.loc[weight_ix, 'unit'] = 'kg'

# print events

In [148]:
df.groupby('feature_name').apply(len)

feature_name
 heart rate / Ventricular Rate    942
% Alpha1                            9
% Alpha2                            9
% Gamma                             9
% LYMPHO                            2
                                 ... 
תרבית מניקור מותניCSF               7
תרגול אקטיבי                        2
תרופות קבועות                       3
תשובת CT חזה בטן אגן                1
תשובת ייעוץ מרפאת כאב               1
Length: 2930, dtype: int64

# Create A Features Table

In [149]:
data = df.loc[df_ix]

In [150]:

before_frames = []
after_frames = []

for index, row in anonymous.loc[anonymous_ix].head().iterrows(): #.head(10)

    #get surgery date of current patient 
    surgery_date = row['Date of surgery']
    
    #print(row['pid'], surgery_date)
    
    #get events before and after date of surgery
    patient = data.loc[data['pid'] == row['pid'], :]
    before = patient[patient['eventStartDate'] < surgery_date]
    after = patient[patient['eventStartDate'] >= surgery_date]
    
    #display(data['eventName'].value_counts())
    
    #calculate statistics
    x1 = before.groupby('feature_name')['numeric'].agg(['mean', 'median', 'min', 'max']) 
    x2 = after.groupby('feature_name')['numeric'].agg([ 'mean', 'median', 'min', 'max'])
    
    #reshape and add pid
    y1 = x1.stack().to_frame().T.assign(pid=row['pid'])
    y2 = x2.stack().to_frame().T.assign(pid=row['pid'])

    #add to list
    before_frames.append(y1)
    after_frames.append(y2)
    
    #display(y1)
    #display(y2)
    
#y1 = pd.concat([x1], keys=['before_surgery'], names=['date'])
#y2 = pd.concat([x2], keys=['after_surgery'], names=['date'])

before_concat = pd.concat(before_frames).set_index('pid', drop=True) 
after_concat = pd.concat(after_frames).set_index('pid', drop=True) 

result_data = pd.concat([before_concat, after_concat], axis=1, 
                        keys=['before_surgery', 'after_surgery'], 
                        names=['date','eventName', 'statistics'])

print("before shape:", before_concat.shape, "Num events:", before_concat.columns.get_level_values(1).nunique())
print("before shape:", after_concat.shape, "Num events:", after_concat.columns.get_level_values(1).nunique())
print("result shape:", result_data.shape)

DataError: No numeric types to aggregate

In [ ]:
result_data

In [ ]:
metadata.columns
#a.merge(res, on='pid').to_csv(DATA_PATH + "/draft1.csv", index=False)

#pd.concat([metadata.columns], keys=['a'], names=['nomi'])

In [ ]:
pd.concat([metadata, result_data], axis=1, levels=['date','eventName', 'statistics'])


In [ ]:
#write output
result.to_excel(DATA_PATH + 'draft1_.xlsx')

## שונות

In [ ]:
def create_feature(df, feature_name, alter_names, column_value='dValue', print_output=True):

    '''
    df - data frame of all events 
    feature_name - name of feature 
    alter_names - alternative names of feature
    
    '''
   
    #get the required events
    sub_events = df[df["eventName"].isin(alter_names)]

    #select only rellevant columns 
    feature = sub_events.loc[:, ['pid', 'eventStartDate', 'eventEndDate', 'unitOfMeasure']]
    
    #get values of event 
    feature[feature_name] = sub_events[column_value]
    
    if print_output:
        
        print("Num {} events:".format(feature_name), sub_events.shape[0])     

        print("\nHas sValue:", sub_events['sValue'].any())
        print("Has event description:", sub_events['eventDesc'].any())

        print("\nDescribe numeric values:\n", sub_events[['dValue', 'iValue']].describe())

        print("\nUnit of measure:", sub_events['unitOfMeasure'].unique())
    
        print("\nResult:")
        display(feature.head())


    return feature


In [ ]:
'''
Adding suffix to duplicate patients (pateints who underwent > 1 surgery) 
'''

res = anonymous.copy()
anonymous.head()
res = res[['pid', 'Sex', 'Age', 'Date of surgery']]

#convert column to strig 
res['pid'] = res['pid'].astype(str)

#Number each item in each group from 1 to the length of that group.
cumcount = res.groupby('pid').cumcount() + 1


#which patients are duplicated
idx = res['pid'].duplicated(keep=False)

#concatenate suffixes
res.loc[idx, 'pid'] = res['pid'] + "_" + cumcount.astype(str)

res[idx];


In [ ]:
ann['cumcount'] = ann.groupby('pid').cumcount()
ann['pid'].astype(str) +"_"+ ann['cumcount'].astype(str)